In [3]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import pillow_heif
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import ipywidgets

In [4]:
test_img_names = set(os.listdir('test'))
train_img_names = set(os.listdir('train/train'))

In [5]:
train_labels_df = pd.read_csv('train/train.csv', sep=';', index_col=None)

In [6]:
train_labels_names = set(train_labels_df['image_name'].values)

## Train / test

In [7]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.classes = [2,5,7]  # car, bus, truk

Using cache found in C:\Users\NDA/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-27 Python-3.9.13 torch-1.12.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [10]:
train_data = []
for img_name in tqdm(train_img_names): 
    if 'heic' in img_name:
        heif_file = pillow_heif.open_heif(os.path.join('train/train/', img_name))
        img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
    else:
        img = Image.open(os.path.join('train/train/', img_name))
    results = model(np.array(img))
    if results.xyxy[0].shape != torch.Size([0, 6]):
        x,y = img.size
        x_rear = x 
        x_median = int(x/2)
        
        area = 1
        index_car = None
        for i in range(len(results.xyxy[0])):
            
            if float(results.xyxy[0][i][0]) < x/2+100 and float(results.xyxy[0][i][2]) > x/2-100:
                
                s = (float(results.xyxy[0][i][2]) - float(results.xyxy[0][i][0]))*(float(results.xyxy[0][i][3]) - float(results.xyxy[0][i][1]))
                if s > area:
                    area = s
                    index_car = i
                    
        a = float(results.xyxy[0][i][2]) - float(results.xyxy[0][i][0])
        b = float(results.xyxy[0][i][3]) - float(results.xyxy[0][i][1])

        if index_car != None:
            
            a = float(results.xyxy[0][index_car][2]) - float(results.xyxy[0][index_car][0])
            b = float(results.xyxy[0][index_car][3]) - float(results.xyxy[0][index_car][1])
            if a/b < 1.7:

                x_min = int(results.xyxy[0][index_car][0])
                y_min = int(results.xyxy[0][index_car][1])
                x_max = int(results.xyxy[0][index_car][2])
                y_max = int(results.xyxy[0][index_car][3])
                results = [img_name] + [x_min, y_min, x_max, y_max]
                train_data.append(results)


100%|████████████████████████████████████████████████████████████████████████████████| 530/530 [05:01<00:00,  1.76it/s]


In [11]:
train_data_df = pd.DataFrame(train_data, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max'])
set(train_data_df['image_name']).symmetric_difference(train_img_names)


{'img_1927.jpg', 'img_2418.jpg', 'img_2733.heic'}

In [12]:
train_data_df

,image_name,x_min,y_min,x_max,y_max
0,img_2503.jpg,1669,1300,2146,1726
1,img_1933.jpg,1457,1034,2268,1858
2,img_2406.jpg,1521,1250,2179,1810
3,img_1853.jpg,1694,1388,2063,1690
4,img_1612.jpg,1720,1349,2201,1830
...,...,...,...,...,...
522,img_2306.jpg,1493,1205,2273,1853
523,img_2430.jpg,1211,936,2530,2057
524,img_2629.jpg,1657,1143,2189,1601
525,img_2452.jpg,1471,1256,2160,1859


In [13]:
train_data_df = pd.merge(train_labels_df, train_data_df, how='left')
train_data_df.to_csv('train.csv', sep=';', index=False)

In [14]:
from catboost import CatBoostRegressor

In [15]:
model_2 = CatBoostRegressor(iterations=None,
                        learning_rate=None,
                        depth=None)

# Fit model
model_2.fit(train_data_df[['x_min', 'y_min', 'x_max', 'y_max']], train_data_df[['distance']].values)

Learning rate set to 0.037035
0:	learn: 1.7067570	total: 153ms	remaining: 2m 32s
1:	learn: 1.6596930	total: 155ms	remaining: 1m 17s
2:	learn: 1.6141172	total: 157ms	remaining: 52.3s
3:	learn: 1.5699267	total: 160ms	remaining: 39.8s
4:	learn: 1.5259575	total: 161ms	remaining: 32s
5:	learn: 1.4856600	total: 163ms	remaining: 26.9s
6:	learn: 1.4467601	total: 164ms	remaining: 23.3s
7:	learn: 1.4089510	total: 166ms	remaining: 20.5s
8:	learn: 1.3735590	total: 167ms	remaining: 18.4s
9:	learn: 1.3390630	total: 168ms	remaining: 16.7s
10:	learn: 1.3061449	total: 170ms	remaining: 15.2s
11:	learn: 1.2749570	total: 171ms	remaining: 14.1s
12:	learn: 1.2417764	total: 173ms	remaining: 13.1s
13:	learn: 1.2094656	total: 174ms	remaining: 12.3s
14:	learn: 1.1807739	total: 176ms	remaining: 11.6s
15:	learn: 1.1526596	total: 177ms	remaining: 10.9s
16:	learn: 1.1283735	total: 179ms	remaining: 10.3s
17:	learn: 1.1012568	total: 180ms	remaining: 9.82s
18:	learn: 1.0772696	total: 182ms	remaining: 9.41s
19:	learn: 

241:	learn: 0.4037231	total: 378ms	remaining: 1.18s
242:	learn: 0.4033051	total: 379ms	remaining: 1.18s
243:	learn: 0.4029180	total: 380ms	remaining: 1.18s
244:	learn: 0.4019688	total: 382ms	remaining: 1.18s
245:	learn: 0.4016243	total: 383ms	remaining: 1.17s
246:	learn: 0.4011875	total: 384ms	remaining: 1.17s
247:	learn: 0.4008476	total: 385ms	remaining: 1.17s
248:	learn: 0.4004689	total: 387ms	remaining: 1.17s
249:	learn: 0.4000692	total: 388ms	remaining: 1.16s
250:	learn: 0.3992432	total: 389ms	remaining: 1.16s
251:	learn: 0.3988856	total: 391ms	remaining: 1.16s
252:	learn: 0.3985514	total: 392ms	remaining: 1.16s
253:	learn: 0.3982538	total: 395ms	remaining: 1.16s
254:	learn: 0.3979119	total: 397ms	remaining: 1.16s
255:	learn: 0.3973138	total: 398ms	remaining: 1.16s
256:	learn: 0.3970494	total: 400ms	remaining: 1.16s
257:	learn: 0.3965321	total: 401ms	remaining: 1.15s
258:	learn: 0.3962119	total: 402ms	remaining: 1.15s
259:	learn: 0.3960179	total: 403ms	remaining: 1.15s
260:	learn: 

447:	learn: 0.3254485	total: 564ms	remaining: 695ms
448:	learn: 0.3249203	total: 565ms	remaining: 694ms
449:	learn: 0.3246442	total: 566ms	remaining: 692ms
450:	learn: 0.3245807	total: 568ms	remaining: 691ms
451:	learn: 0.3242806	total: 569ms	remaining: 690ms
452:	learn: 0.3239023	total: 570ms	remaining: 688ms
453:	learn: 0.3238362	total: 571ms	remaining: 687ms
454:	learn: 0.3234725	total: 573ms	remaining: 686ms
455:	learn: 0.3230426	total: 574ms	remaining: 685ms
456:	learn: 0.3226383	total: 575ms	remaining: 684ms
457:	learn: 0.3223895	total: 576ms	remaining: 682ms
458:	learn: 0.3221351	total: 578ms	remaining: 681ms
459:	learn: 0.3219314	total: 579ms	remaining: 680ms
460:	learn: 0.3217868	total: 580ms	remaining: 678ms
461:	learn: 0.3213575	total: 581ms	remaining: 677ms
462:	learn: 0.3212101	total: 582ms	remaining: 675ms
463:	learn: 0.3206844	total: 583ms	remaining: 674ms
464:	learn: 0.3202799	total: 584ms	remaining: 672ms
465:	learn: 0.3196319	total: 585ms	remaining: 670ms
466:	learn: 

674:	learn: 0.2639443	total: 763ms	remaining: 367ms
675:	learn: 0.2633806	total: 765ms	remaining: 366ms
676:	learn: 0.2630121	total: 766ms	remaining: 365ms
677:	learn: 0.2627766	total: 767ms	remaining: 364ms
678:	learn: 0.2624963	total: 767ms	remaining: 363ms
679:	learn: 0.2623342	total: 769ms	remaining: 362ms
680:	learn: 0.2621840	total: 770ms	remaining: 361ms
681:	learn: 0.2620081	total: 771ms	remaining: 360ms
682:	learn: 0.2617579	total: 772ms	remaining: 358ms
683:	learn: 0.2616214	total: 773ms	remaining: 357ms
684:	learn: 0.2614284	total: 775ms	remaining: 356ms
685:	learn: 0.2613529	total: 776ms	remaining: 355ms
686:	learn: 0.2611579	total: 777ms	remaining: 354ms
687:	learn: 0.2608829	total: 779ms	remaining: 353ms
688:	learn: 0.2605932	total: 780ms	remaining: 352ms
689:	learn: 0.2602017	total: 782ms	remaining: 351ms
690:	learn: 0.2598887	total: 785ms	remaining: 351ms
691:	learn: 0.2596073	total: 786ms	remaining: 350ms
692:	learn: 0.2593231	total: 787ms	remaining: 349ms
693:	learn: 

893:	learn: 0.2234702	total: 962ms	remaining: 114ms
894:	learn: 0.2231895	total: 963ms	remaining: 113ms
895:	learn: 0.2230782	total: 965ms	remaining: 112ms
896:	learn: 0.2229175	total: 966ms	remaining: 111ms
897:	learn: 0.2227679	total: 967ms	remaining: 110ms
898:	learn: 0.2226848	total: 969ms	remaining: 109ms
899:	learn: 0.2225057	total: 970ms	remaining: 108ms
900:	learn: 0.2224846	total: 972ms	remaining: 107ms
901:	learn: 0.2224704	total: 972ms	remaining: 106ms
902:	learn: 0.2222055	total: 974ms	remaining: 105ms
903:	learn: 0.2220799	total: 975ms	remaining: 104ms
904:	learn: 0.2220699	total: 976ms	remaining: 102ms
905:	learn: 0.2218417	total: 977ms	remaining: 101ms
906:	learn: 0.2218000	total: 978ms	remaining: 100ms
907:	learn: 0.2215861	total: 980ms	remaining: 99.3ms
908:	learn: 0.2214733	total: 981ms	remaining: 98.2ms
909:	learn: 0.2214359	total: 982ms	remaining: 97.1ms
910:	learn: 0.2212810	total: 984ms	remaining: 96.2ms
911:	learn: 0.2211854	total: 985ms	remaining: 95.1ms
912:	le

In [19]:
test_data = []

for img_name in tqdm(test_img_names): 
    if 'heic' in img_name:
        heif_file = pillow_heif.open_heif(os.path.join('test/', img_name))
        img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
    else:
        img = Image.open(os.path.join('test/', img_name))
    results = model(np.array(img))
    if results.xyxy[0].shape != torch.Size([0, 6]):
        x,y = img.size
        area = 1
        index_car = None
        for i in range(len(results.xyxy[0])):
            
            if float(results.xyxy[0][i][0]) < x/2+100 and float(results.xyxy[0][i][2]) > x/2-100:
                
                s = (float(results.xyxy[0][i][2]) - float(results.xyxy[0][i][0]))*(float(results.xyxy[0][i][3]) - float(results.xyxy[0][i][1]))
                if s > area:
                    area = s
                    index_car = i

        if index_car != None:
            x_min = int(results.xyxy[0][index_car][0])
            y_min = int(results.xyxy[0][index_car][1])
            x_max = int(results.xyxy[0][index_car][2])
            y_max = int(results.xyxy[0][index_car][3])
            results = [img_name] + [x_min, y_min, x_max, y_max]
            test_data.append(results)
        else:
            x_min = 0
            y_min = 0
            x_max = 0
            y_max = 0
            results = [img_name] + [x_min, y_min, x_max, y_max]
            test_data.append(results)
            index_car = 0

    else:
        x_min = 0
        y_min = 0
        x_max = 0
        y_max = 0
        results = [img_name] + [x_min, y_min, x_max, y_max]
        test_data.append(results)


100%|████████████████████████████████████████████████████████████████████████████████| 521/521 [05:14<00:00,  1.65it/s]

521


In [20]:
test_data_df = pd.DataFrame(test_data, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max'])
test_data_df

,image_name,x_min,y_min,x_max,y_max
0,img_2312.jpg,1649,1198,2057,1528
1,img_2371.jpg,1488,1041,2506,1908
2,img_1843.jpg,1550,1210,2249,1789
3,img_2609.jpg,1023,913,2790,2446
4,img_2407.jpg,1516,1120,2262,1782
...,...,...,...,...,...
516,img_1644.jpg,1609,1203,2175,1702
517,img_2719.heic,1698,1363,2138,1723
518,img_1947.jpg,1545,1221,2270,1806
519,img_1663.jpg,1566,872,2170,1368


In [21]:
for i in range(len(test_data_df['image_name'])):
    if test_data_df.loc[i, 'x_min'] == 0 and test_data_df.loc[i, 'y_min']==0 and test_data_df.loc[i, 'x_max'] == 0 and test_data_df.loc[i, 'y_max'] == 0:
        test_data_df.loc[i, 'x_min'] = int(test_data_df['x_min'].median())
        test_data_df.loc[i, 'y_min'] = int(test_data_df['y_min'].median())
        test_data_df.loc[i, 'x_max'] = int(test_data_df['x_max'].median())
        test_data_df.loc[i, 'y_max'] = int(test_data_df['y_max'].median())
test_data_df.to_csv('test.csv', sep=';', index=False)

464
469


In [22]:
preds = model_2.predict(test_data_df[['x_min', 'y_min', 'x_max', 'y_max']])

In [23]:
test_data_df['distance'] = preds

In [24]:
sample_solution_df = test_data_df[['image_name', 'distance']]

In [26]:
lost_test_items = []

for file_name in test_img_names - set(sample_solution_df['image_name'].values):
    lost_test_items.append([file_name, 0])
lost_test_items

[]

In [27]:
lost_test_items_df = pd.DataFrame(lost_test_items, columns=['image_name', 'distance'])


,image_name,distance


In [28]:
sample_solution_df = pd.concat([sample_solution_df, lost_test_items_df])


In [29]:
sample_solution_df.to_csv('sample_solution_27-08-22.csv', sep=';', index=False)